<a href="https://colab.research.google.com/github/marina554/accounting-practice/blob/main/Standard_Costing_Materials%2C_Labor_%26_Manufacturing_Overhead_(Variable_%2B_Fixed).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# ===============================
# Variance Engine (Prototype)
# ===============================

class MaterialLaborInput:
    """
    Container for standard vs actual values for materials or labor.
    """
    def __init__(self, std_price, std_qty, actual_price, actual_qty):
        self.std_price = std_price
        self.std_qty = std_qty
        self.actual_price = actual_price
        self.actual_qty = actual_qty


class OverheadInput:
    """
    Input class for manufacturing overhead (variable + fixed).
    """
    def __init__(
        self,
        std_rate_var,
        std_rate_fix,
        std_hours,
        actual_rate_var,
        actual_rate_fix,
        actual_hours,
        actual_var_cost,
        actual_fix_cost,
        normal_capacity_hours
    ):
        self.std_rate_var = std_rate_var
        self.std_rate_fix = std_rate_fix
        self.std_hours = std_hours
        self.actual_rate_var = actual_rate_var
        self.actual_rate_fix = actual_rate_fix
        self.actual_hours = actual_hours
        self.actual_var_cost = actual_var_cost
        self.actual_fix_cost = actual_fix_cost
        self.normal_capacity_hours = normal_capacity_hours


# ===============================
# Material / Labor Variances
# ===============================

class MaterialLaborVariance:

    @staticmethod
    def price_variance(data: MaterialLaborInput):
        return (data.actual_price - data.std_price) * data.actual_qty

    @staticmethod
    def quantity_variance(data: MaterialLaborInput):
        return (data.actual_qty - data.std_qty) * data.std_price

    @staticmethod
    def total_variance(data: MaterialLaborInput):
        return (
            MaterialLaborVariance.price_variance(data)
            + MaterialLaborVariance.quantity_variance(data)
        )


# ===============================
# Manufacturing Overhead Variances
# ===============================

class OverheadVariance:

    @staticmethod
    def variable_spending_variance(data: OverheadInput):
        return data.actual_var_cost - (data.std_rate_var * data.actual_hours)

    @staticmethod
    def variable_efficiency_variance(data: OverheadInput):
        return (data.actual_hours - data.std_hours) * data.std_rate_var

    @staticmethod
    def fixed_budget_variance(data: OverheadInput):
        return data.actual_fix_cost - (data.std_rate_fix * data.normal_capacity_hours)

    @staticmethod
    def fixed_volume_variance(data: OverheadInput):
        return (data.std_hours - data.normal_capacity_hours) * data.std_rate_fix

    @staticmethod
    def total_overhead_variance(data: OverheadInput):
        return (
            OverheadVariance.variable_spending_variance(data)
            + OverheadVariance.variable_efficiency_variance(data)
            + OverheadVariance.fixed_budget_variance(data)
            + OverheadVariance.fixed_volume_variance(data)
        )


# ===============================
# Sample Data
# ===============================

mat = MaterialLaborInput(
    std_price=5000,
    std_qty=100,
    actual_price=5500,
    actual_qty=120
)

oh = OverheadInput(
    std_rate_var=200,
    std_rate_fix=300,
    std_hours=100,
    actual_rate_var=220,
    actual_rate_fix=300,
    actual_hours=120,
    actual_var_cost=24000,
    actual_fix_cost=35000,
    normal_capacity_hours=110
)


# ===============================
# Variance Table (DataFrame)
# ===============================

variance_table = pd.DataFrame({
    "Variance Item": [
        "Material/Labor – Price Variance",
        "Material/Labor – Quantity Variance",
        "Material/Labor – Total Variance",
        "Variable OH – Spending Variance",
        "Variable OH – Efficiency Variance",
        "Fixed OH – Budget Variance",
        "Fixed OH – Volume Variance",
        "Total Manufacturing OH Variance",
    ],
    "Amount": [
        MaterialLaborVariance.price_variance(mat),
        MaterialLaborVariance.quantity_variance(mat),
        MaterialLaborVariance.total_variance(mat),
        OverheadVariance.variable_spending_variance(oh),
        OverheadVariance.variable_efficiency_variance(oh),
        OverheadVariance.fixed_budget_variance(oh),
        OverheadVariance.fixed_volume_variance(oh),
        OverheadVariance.total_overhead_variance(oh),
    ]
})

print("=== Variance Summary (Table) ===")
print(variance_table)


=== Variance Summary (Table) ===
                        Variance Item  Amount
0     Material/Labor – Price Variance   60000
1  Material/Labor – Quantity Variance  100000
2     Material/Labor – Total Variance  160000
3     Variable OH – Spending Variance       0
4   Variable OH – Efficiency Variance    4000
5          Fixed OH – Budget Variance    2000
6          Fixed OH – Volume Variance   -3000
7     Total Manufacturing OH Variance    3000


In [6]:
import pandas as pd

# ===============================
# 差異計算エンジン（プロトタイプ）
# ===============================

class MaterialLaborInput:
    """
    材料費・労務費の標準値と実際値を格納するクラス
    """
    def __init__(self, std_price, std_qty, actual_price, actual_qty):
        self.std_price = std_price       # 標準単価
        self.std_qty = std_qty           # 標準数量
        self.actual_price = actual_price # 実際単価
        self.actual_qty = actual_qty     # 実際数量


class OverheadInput:
    """
    製造間接費（変動 + 固定）の入力データ用クラス
    """
    def __init__(
        self,
        std_rate_var,      # 変動間接費の標準配賦率
        std_rate_fix,      # 固定間接費の標準配賦率
        std_hours,         # 標準作業時間
        actual_rate_var,
        actual_rate_fix,
        actual_hours,
        actual_var_cost,   # 実際発生した変動間接費
        actual_fix_cost,   # 実際発生した固定間接費
        normal_capacity_hours  # 正常操業度（固定費配賦の基準）
    ):
        self.std_rate_var = std_rate_var
        self.std_rate_fix = std_rate_fix
        self.std_hours = std_hours
        self.actual_rate_var = actual_rate_var
        self.actual_rate_fix = actual_rate_fix
        self.actual_hours = actual_hours
        self.actual_var_cost = actual_var_cost
        self.actual_fix_cost = actual_fix_cost
        self.normal_capacity_hours = normal_capacity_hours


# ===============================
# 材料費・労務費の差異
# ===============================

class MaterialLaborVariance:

    @staticmethod
    def price_variance(data: MaterialLaborInput):
        # 価格差異 = (実際単価 - 標準単価) × 実際数量
        return (data.actual_price - data.std_price) * data.actual_qty

    @staticmethod
    def quantity_variance(data: MaterialLaborInput):
        # 数量差異 = (実際数量 - 標準数量) × 標準単価
        return (data.actual_qty - data.std_qty) * data.std_price

    @staticmethod
    def total_variance(data: MaterialLaborInput):
        # 総差異 = 価格差異 + 数量差異
        return (
            MaterialLaborVariance.price_variance(data)
            + MaterialLaborVariance.quantity_variance(data)
        )


# ===============================
# 製造間接費の差異（変動 + 固定）
# ===============================

class OverheadVariance:

    # --- 変動製造間接費 ---
    @staticmethod
    def variable_spending_variance(data: OverheadInput):
        # 変動費支払差異 = 実際変動費 - (標準変動配賦率 × 実際作業時間)
        return data.actual_var_cost - (data.std_rate_var * data.actual_hours)

    @staticmethod
    def variable_efficiency_variance(data: OverheadInput):
        # 能率差異 = (実際作業時間 - 標準作業時間) × 標準変動配賦率
        return (data.actual_hours - data.std_hours) * data.std_rate_var

    # --- 固定製造間接費 ---
    @staticmethod
    def fixed_budget_variance(data: OverheadInput):
        # 予算差異 = 実際固定費 - (標準固定配賦率 × 正常操業度)
        return data.actual_fix_cost - (data.std_rate_fix * data.normal_capacity_hours)

    @staticmethod
    def fixed_volume_variance(data: OverheadInput):
        # 操業度差異 = (標準作業時間 - 正常操業度) × 標準固定配賦率
        return (data.std_hours - data.normal_capacity_hours) * data.std_rate_fix

    @staticmethod
    def total_overhead_variance(data: OverheadInput):
        return (
            OverheadVariance.variable_spending_variance(data)
            + OverheadVariance.variable_efficiency_variance(data)
            + OverheadVariance.fixed_budget_variance(data)
            + OverheadVariance.fixed_volume_variance(data)
        )


# ===============================
# サンプルデータ
# ===============================

# 材料：標準 5,000円 × 100個、実際 5,500円 × 120個
mat = MaterialLaborInput(
    std_price=5000,
    std_qty=100,
    actual_price=5500,
    actual_qty=120
)

# 製造間接費（サンプル）
oh = OverheadInput(
    std_rate_var=200,
    std_rate_fix=300,
    std_hours=100,
    actual_rate_var=220,
    actual_rate_fix=300,
    actual_hours=120,
    actual_var_cost=24000,
    actual_fix_cost=35000,
    normal_capacity_hours=110
)


# ===============================
# 差異一覧を表形式でまとめる
# ===============================

variance_table = pd.DataFrame({
    "差異項目": [
        "材料・労務：価格差異",
        "材料・労務：数量差異",
        "材料・労務：総差異",
        "変動間接費：支払差異",
        "変動間接費：能率差異",
        "固定間接費：予算差異",
        "固定間接費：操業度差異",
        "総間接費差異",
    ],
    "金額": [
        MaterialLaborVariance.price_variance(mat),
        MaterialLaborVariance.quantity_variance(mat),
        MaterialLaborVariance.total_variance(mat),
        OverheadVariance.variable_spending_variance(oh),
        OverheadVariance.variable_efficiency_variance(oh),
        OverheadVariance.fixed_budget_variance(oh),
        OverheadVariance.fixed_volume_variance(oh),
        OverheadVariance.total_overhead_variance(oh),
    ]
})

print("=== 差異一覧（表形式） ===")
print(variance_table)


=== 差異一覧（表形式） ===
          差異項目      金額
0   材料・労務：価格差異   60000
1   材料・労務：数量差異  100000
2    材料・労務：総差異  160000
3   変動間接費：支払差異       0
4   変動間接費：能率差異    4000
5   固定間接費：予算差異    2000
6  固定間接費：操業度差異   -3000
7       総間接費差異    3000
